In [0]:
%sql
CREATE OR REPLACE TABLE gold.dim_product_category (
  -- Incremental surrogate key
  _tf_dim_product_id BIGINT GENERATED BY DEFAULT AS IDENTITY (START WITH 1 INCREMENT BY 1) PRIMARY KEY NOT NULL,

  -- Attributes
  prod_product_id INT,                   -- Clé naturelle issue de la source
  prod_name STRING,                      -- Nom du produit
  prod_product_number STRING,            -- Numéro de produit
  prod_color STRING,                     -- Couleur du produit
  prod_size STRING,                      -- Taille du produit
  prod_standard_cost DECIMAL(18, 2),     -- Coût standard
  prod_list_price DECIMAL(18, 2),        -- Prix de liste
  prod_weight DECIMAL(18, 2),            -- Poids du produit
  prod_model_name STRING,                -- Nom du modèle
  prod_category_name_l1 STRING,          -- Nom de la catégorie level 1
  prod_category_name_l2 STRING,          -- Nom de la catégorie level 2
  prod_thumbnail_photo STRING,           -- Miniature du produit
  prod_description STRING,               -- Description du produit

  -- Technical columns
  _tf_create_date TIMESTAMP,             -- Date de création
  _tf_update_date TIMESTAMP              -- Date de mise à jour
);

In [0]:
%sql
USE CATALOG jeromeattinger_lakehouse;
USE SCHEMA gold;

DECLARE OR REPLACE load_date = current_timestamp();
VALUES load_date;

In [0]:
%sql
MERGE INTO gold.dim_product_category AS tgt
USING (
    SELECT
        CAST(p.product_id AS INT) AS prod_product_id,
        COALESCE(TRY_CAST(p.name AS STRING), 'N/A') AS prod_name,
        COALESCE(TRY_CAST(p.product_number AS STRING), 'N/A') AS prod_product_number,
        COALESCE(TRY_CAST(p.color AS STRING), 'N/A') AS prod_color,
        COALESCE(TRY_CAST(p.size AS STRING), 'N/A') AS prod_size,
        COALESCE(TRY_CAST(p.standard_cost AS DECIMAL(18, 2)), 0.00) AS prod_standard_cost,
        COALESCE(TRY_CAST(p.list_price AS DECIMAL(18, 2)), 0.00) AS prod_list_price,
        COALESCE(TRY_CAST(p.weight AS DECIMAL(18, 2)), 0.00) AS prod_weight,
        COALESCE(TRY_CAST(pm.name AS STRING), 'N/A') AS prod_model_name, -- Nom du modèle

        -- Gestion des catégories
        COALESCE(TRY_CAST(parent_category.Name AS STRING), 'N/A') AS prod_category_name_l1, -- Catégorie niveau 1
        COALESCE(TRY_CAST(child_category.Name AS STRING), 'N/A') AS prod_category_name_l2, -- Catégorie niveau 2

        COALESCE(TRY_CAST(p.thumbnail_photo AS STRING), 'N/A') AS prod_thumbnail_photo,
        COALESCE(TRY_CAST(pd.description AS STRING), 'N/A') AS prod_description, -- Description

        current_timestamp() AS _tf_create_date,
        current_timestamp() AS _tf_update_date

    FROM silver.product AS p
    LEFT JOIN silver.productmodel AS pm
        ON p.product_model_id = pm.product_model_id
    LEFT JOIN silver.productcategory AS child_category
        ON p.product_category_id = child_category.product_category_id
    LEFT JOIN silver.productcategory AS parent_category
        ON child_category.parent_product_category_id = parent_category.product_category_id
    LEFT JOIN silver.productmodelproductdescription AS pmpd
        ON p.product_model_id = pmpd.product_model_id
    LEFT JOIN silver.productdescription AS pd
        ON pmpd.product_description_id = pd.product_description_id
    WHERE pmpd.culture = 'en' -- Filtrer pour les descriptions en anglais
      AND p._tf_valid_to IS NULL
) AS src
ON tgt.prod_product_id = src.prod_product_id

-- 1) Mettre à jour les enregistrements existants si des différences sont détectées
WHEN MATCHED AND (
    tgt.prod_name != src.prod_name OR
    tgt.prod_product_number != src.prod_product_number OR
    tgt.prod_color != src.prod_color OR
    tgt.prod_size != src.prod_size OR
    tgt.prod_standard_cost != src.prod_standard_cost OR
    tgt.prod_list_price != src.prod_list_price OR
    tgt.prod_weight != src.prod_weight OR
    tgt.prod_model_name != src.prod_model_name OR
    tgt.prod_category_name_l1 != src.prod_category_name_l1 OR
    tgt.prod_category_name_l2 != src.prod_category_name_l2 OR
    tgt.prod_thumbnail_photo != src.prod_thumbnail_photo OR
    tgt.prod_description != src.prod_description
) THEN
  UPDATE SET 
    tgt.prod_name = src.prod_name,
    tgt.prod_product_number = src.prod_product_number,
    tgt.prod_color = src.prod_color,
    tgt.prod_size = src.prod_size,
    tgt.prod_standard_cost = src.prod_standard_cost,
    tgt.prod_list_price = src.prod_list_price,
    tgt.prod_weight = src.prod_weight,
    tgt.prod_model_name = src.prod_model_name,
    tgt.prod_category_name_l1 = src.prod_category_name_l1,
    tgt.prod_category_name_l2 = src.prod_category_name_l2,
    tgt.prod_thumbnail_photo = src.prod_thumbnail_photo,
    tgt.prod_description = src.prod_description,
    tgt._tf_update_date = current_timestamp()

-- 2) Insérer de nouveaux enregistrements
WHEN NOT MATCHED THEN
  INSERT (
    prod_product_id,
    prod_name,
    prod_product_number,
    prod_color,
    prod_size,
    prod_standard_cost,
    prod_list_price,
    prod_weight,
    prod_model_name,
    prod_category_name_l1,
    prod_category_name_l2,
    prod_thumbnail_photo,
    prod_description,
    _tf_create_date,
    _tf_update_date
  )
  VALUES (
    src.prod_product_id,
    src.prod_name,
    src.prod_product_number,
    src.prod_color,
    src.prod_size,
    src.prod_standard_cost,
    src.prod_list_price,
    src.prod_weight,
    src.prod_model_name,
    src.prod_category_name_l1,
    src.prod_category_name_l2,
    src.prod_thumbnail_photo,
    src.prod_description,
    src._tf_create_date,
    src._tf_update_date
  );